<a href="https://colab.research.google.com/github/adlermferreira/Chatbot_project/blob/master/train_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>